In [1]:
import traceback
import datetime
import yaml

import pytz
from Gradio_UI import GradioUI
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, load_tool, tool

from ollama.smol_compatiable_model import OllamaQwen7BClientModel
from tools.final_answer import FinalAnswerTool

/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@tool
def search_online(search_query:str, max_results:int) -> str: 
    """A tool that gets info from the internet (via DuckDuckGo) and prints it out in a single string.
    Args:
        search_query: the first argument
        max_results: the second argument
    """
    try:
        resp = DuckDuckGoSearchTool().forward(search_query, max_results=max_results)
    except Exception as e:
        resp = "Could not use the websearch tool, something went wrong: " + traceback.format_exc() # or e.__repr__
    return resp

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

In [3]:
final_answer = FinalAnswerTool()

In [4]:

# model = InferenceClientModel(
#     max_tokens=2096,
#     temperature=0.5,
#     model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
#     custom_role_conversions=None,
# )
model = OllamaQwen7BClientModel()


# Import tool from Hub
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

In [5]:
with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)    
prompt_templates["final_answer"] =  {"{final_answer}"}

print(prompt_templates)

{'system_prompt': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using the `final_ans

In [6]:
prompt_templates = {
    "system": {
        "prefix": "You are a coding assistant. Follow rules:\n- Use tools\n- Explain steps.",
        "suffix": ""
    },
    "user": {
        "prefix": "Task: {input}",
        "suffix": ""
    },
    "tool": {
        "prefix": "Action: Run {tool_name} with input: {tool_input}",
        "suffix": ""
    },
    "tool_result": {
        "prefix": "Result: {tool_output}",
        "suffix": ""
    },
    "final_answer": {
        "prefix": "Final Answer: {final_answer}",
        "suffix": ""
    }
    
}

In [8]:
agent = CodeAgent(
    model=model,
    tools=[final_answer, get_current_time_in_timezone, search_online], # add your tools here (don't remove final_answer)
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    # prompt_templates=prompt_templates # Pass system prompt to CodeAgent
)


In [10]:
agent.run("Just get some info about Prussia")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Just get some info about Prussia                                                                                │
│                                                                                                                 │
╰─ OllamaQwen7BClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ─────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
404 page not found

[Step 1: Duration 0.04 seconds]

AgentGenerationError: Error in generating model output:
404 page not found